# 智能体(Agent)
ERNIE Bot Agent 基于文心大模型的 Function Calling 能力实现了多工具编排和自动调度功能，并且允许工具、插件、知识库等不同组件的混合编排。

除了自动调度，我们未来还将支持更多的编排模式，例如手动编排、半自动编排，为开发者提供更大的灵活性。



# Function Agent介绍
在此教程中，主要介绍一个自动智能调度的Function Agent的运行过程，Function Agent的编排能力由集成的LLM大语言模型（此处为文心）的函数调用能力驱动。

通常由Agent将用户的自然语言输入传递请求LLM生成一个可以解析的Action，

然后Agent根据Action，调用相应的工具或插件，然后完成此操作并再次请求LLM决定是否要产生下一个Action或者终止。

一个智能体的运行大致分为以下三步：
1. 创建需要调用的工具
2. 创建智能体
3. 调用智能体

[Optional]建议将环境变量`EB_AGENT_LOGGING_LEVEL`设置为info打开相关日志，以方便观察智能体具体的运行过程。


接下来将按照这三步，结合一个具体的实例进行介绍：

实例的背景为创建一个英语学习智能体，帮助用户学习英语单词，包括单词的释义、近义词、反义词和例句并将相关信息添加到单词本。

In [1]:
import os

os.environ["EB_AGENT_LOGGING_LEVEL"] = "INFO"
os.environ["EB_AGENT_ACCESS_TOKEN"] = "<access_token>"

from pydantic import Field
from typing import Dict, Type, List, Any

from erniebot_agent.chat_models import ERNIEBot
from erniebot_agent.memory import HumanMessage, Message, AIMessage, FunctionMessage, SlidingWindowMemory
from erniebot_agent.tools.base import Tool
from erniebot_agent.tools.schema import ToolParameterView
from erniebot_agent.agents import FunctionAgent
from erniebot_agent.tools import RemoteToolkit
from erniebot_agent.file import GlobalFileManagerHandler

## 创建工具
此章节中，主要创建用于英语学习智能体相关的本地工具以便于用户直接体验，不会对tool的相关内容有太多详细的介绍，关于tool的详细内容，详见[local_tool.ipynb](./local_tool.ipynb)。

其中，对于工具的examples的描写，用到了Message模块，详见[memory.ipynb](./memory.ipynb)。

## 定义ChatWithEB工具
在智能体(Agent)中，我们创建一个名为ChatWithEB的工具，该工具用于为用户的单词提供详细的释义，其中可以通过`prompt`或者`system`的方式来控制，达成用户的相关目标（本例中为按要求获取单词相关信息）。

同时`ChatWithEB`工具也是一个推荐的本地工具，加载在智能体当中用于每一个Run中途的交互，从而更好地完成智能体的功能。

例如在本例中，我们通过`ChatWithEB`工具来获取用户输入的单词的详细信息，进而能直接添加进入单词本当中。

如果不使用这个工具，那么用户无法通过一句简单的"帮我添加单词"同时执行这三个操作：

需要先通过OCR获得单词内容，然后开启下一轮对话通过prompt获得单词相关内容，然后再通过"帮我添加单词"添加相关单词。


In [2]:
class ChatWithEBInputView(ToolParameterView):
    word: str = Field(description="需要获取详细信息的单词")


class ChatWithEBOutputView(ToolParameterView):
    response: str = Field(description="单词的详细释义")


class ChatWithEB(Tool):
    description: str = "ChatWithEB是一款根据用户的提供的单词，获取一个具体的单词描述、翻译以及例句的工具"
    input_type: Type[ToolParameterView] = ChatWithEBInputView
    ouptut_type: Type[ToolParameterView] = ChatWithEBOutputView

    def __init__(self, llm: ERNIEBot):
        self.llm = llm

    async def __call__(self, word: str) -> Dict[str, str]:
        prompt = f"请告诉我{word}的详细信息，以下面的格式输出（其中单词翻译需要是中文，其他为英语）" "<单词翻译>:\n<近义词>:\n<反义词>:\n<例句>:\n"
        response = await self.llm.chat([HumanMessage(prompt)])
        return {"response": response.content}

    @property
    def examples(self) -> List[Message]:
        return [
            HumanMessage("请帮我把这个单词meticulous存储到单词本中"),
            AIMessage(
                "",
                function_call={
                    "name": self.tool_name,
                    "thoughts": f"用户想把meticulous加入到单词本，我需要先使用{self.tool_name}来获取"
                    "单词'meticulous'的详细信息，然后调用AddWordTool来存储到单词本中",
                    "arguments": '{"word": "meticulous"}',
                },
            ),
        ]


SYSTEM_MESSAGE = "你是一个英语老师，当用户给你一个单词的时候，请你专业而通俗的返回英语的<单词翻译><近义词><反义词><例句>"
llm = ERNIEBot(model="ernie-4.0", system=SYSTEM_MESSAGE)
word_helper_tool = ChatWithEB(llm)

## 定义AddWordTool工具
在此智能体(Agent)中，我们创建一个名为AddWordTool的工具，用于将用户的单词添加至单词本。此工具为`Tool`章节中的示例工具，此处不做过多介绍。详见[local_tool.ipynb](./local_tool.ipynb)。

In [3]:
mimic_response = (
    "<单词翻译>: meticulous中文翻译为“一丝不苟的”、“细致的”、“小心翼翼的”。\n\n"
    "<近义词>: thorough, detailed, precise, conscientious\n\n"
    "<反义词>: careless, sloppy, negligent, cursory\n\n"
    "<例句>: He is known for his meticulous attention to detail in his work."
)


class AddWordInput(ToolParameterView):
    word: str = Field(description="待添加的单词")
    des: str = Field(description="待添加的单词的详细释义")


class AddWordOutput(ToolParameterView):
    result: str = Field(description="表示是否成功将单词成功添加到词库当中")


class AddWordTool(Tool):
    description: str = "添加单词以及它的详细解释到词库当中"
    input_type: Type[ToolParameterView] = AddWordInput
    ouptut_type: Type[ToolParameterView] = AddWordOutput

    def __init__(self) -> None:
        self.word_books = {}
        super().__init__()

    async def __call__(self, word: str, des: str) -> Dict[str, Any]:
        if word in self.word_books:
            return {"result": f"<{word}>单词已经存在，无需添加"}
        self.word_books[word] = des
        words = "\n".join(list(self.word_books.keys()))
        return {"result": f"<{word}>单词已添加成功, 当前单词本中有如下单词：{words}，释义如下{des}"}

    @property
    def examples(self) -> List[Message]:
        return [
            HumanMessage("请帮我把图片中的单词存储到单词本中"),
            AIMessage(
                "",
                function_call={
                    "name": self.tool_name,
                    "thoughts": f"用户需要识别图片中的文字，并解释存储到单词本中，我需要先使用OCR工具识别图片中的文字，然后使用ChatWithEB工具进行解释，最后使用AddWordTool工具存储到单词本中。",
                    "arguments": '{"image":"file-local-5255942c-a55c-11ee-b46a-56c70f692df9"}',
                },
            ),
            FunctionMessage(content='{"result": "meticulous"}', name="OCR工具"),
            AIMessage(
                "",
                function_call={
                    "name": self.tool_name,
                    "thoughts": f"已经从图片中识别出了meticulous，接下来我需要使用ChatWithEB工具进行解释，获取单词'meticulous'的详细信息，"
                    "然后调用AddWordTool来存储到单词本中",
                    "arguments": '{"word": "meticulous"}',
                },
            ),
            FunctionMessage(content=f'{{"response":{mimic_response}}}', name="ChatWithEB"),
            AIMessage(
                "",
                function_call={
                    "name": self.tool_name,
                    "thoughts": f"用户想把meticulous加入到单词本，我已经使用ChatWithEB来获取单词'meticulous'的详细信息，然后需要调用{self.tool_name}来存储到单词本中，单词的具体信息需要把ChatWithEB的response填入'des'中",
                    "arguments": f'{{"word": "meticulous", "des":{mimic_response}}}',
                },
            ),
        ]


add_word_tool = AddWordTool()

## 从AIStudio导入RemoteTool
此处不做过多介绍。详见[remote_tool.ipynb](./remote_tool.ipynb)。

In [4]:
ocr_tool = RemoteToolkit.from_aistudio("pp-ocrv4").get_tools()[0]

## 创建智能体
在此步骤中将展示如何创建智能体。

1. 指定调用的文心大模型，通过`enable_multi_step_tool_call`接口打开multi_tool_call功能，开启之后即可通过一个指令进行多次不同工具的调用，通过`max_steps`控制此Run中执行的最大的Step以免一直循环。

2. 为智能体添加默认的`ChatWithEB`和`AddWordTool`工具。

3. [Optional]为智能体指定特殊的`Memory`，默认为`WholeMemory`，详见[memory.ipynb](./memory.ipynb)。

4. [Optional]为智能体指定`SystemMessage`，设置全局的指令。

In [5]:
memory = SlidingWindowMemory(max_round=5)
llm_final = ERNIEBot(model="ernie-3.5", api_type="aistudio", enable_multi_step_tool_call=True)
agent_all = FunctionAgent(llm=llm_final, tools=[word_helper_tool, add_word_tool, ocr_tool], memory=memory)

## 调用智能体
在完成了智能体创建之后，通过异步run函数执行指令，并获取返回结果（如果打开了multi_tool_call，那么仅需一句指令即可同时执行图片OCR，单词解释以及添加至单词本三个动作）。

如果需要运行过程中涉及到文件，可以先通过`file_manager`创建`File`对象，再通过run的`files`参数传入。

In [7]:
file_manager = GlobalFileManagerHandler().get()
file = await file_manager.create_file_from_path(r"./ocr.png")
response = await agent_all.run("请帮我把图片中的单词进行解释后存储到单词本中", files=[file])

INFO - [Run][Start] FunctionAgent is about to start running with input:
请帮我把图片中的单词进行解释后存储到单词本中
INFO - [LLM][Start] ERNIEBot is about to start running with input:
 role: user 
 content: 请帮我把图片中的单词进行解释后存储到单词本中
<file>file-local-794791f0-a55e-11ee-b9cd-56c70f692df9</file> 
INFO - [LLM][End] ERNIEBot finished running with output:
 role: assistant 
 function_call: 
{
  "name": "pp-ocrv4/v2.2/ocr",
  "thoughts": "用户需要识别图片中的文字，并解释存储到单词本中，我需要先使用OCR工具识别图片中的文字，然后使用ChatWithEB工具进行解释，最后使用AddWordTool工具存储到单词本中。",
  "arguments": "{\"image\":\"file-local-794791f0-a55e-11ee-b9cd-56c70f692df9\"}"
} 
INFO - [Tool][Start] RemoteTool is about to start running with input:
{
  "image": "file-local-794791f0-a55e-11ee-b9cd-56c70f692df9"
}
INFO - [Tool][End] RemoteTool finished running with output:
{
  "result": "meticulous"
}
INFO - [LLM][Start] ERNIEBot is about to start running with input:
 role: function 
 name: pp-ocrv4/v2.2/ocr 
 content: {"result": "meticulous"} 
INFO - [LLM][End] ERNIEBot finished running

In [9]:
print(response.text)

您已经将单词“meticulous”添加到单词本中，以下是它的详细解释和相关信息：

<单词翻译>: meticulous是“一丝不苟的，细致的，小心翼翼的”意思。

<近义词>:\n1. thorough - 彻底的，详尽的\n2. careful - 小心的，仔细的\n3. precise - 精确的，严谨的\n4. attention to detail - 关注细节

<反义词>:\n1. careless - 粗心的，不仔细的\n2. sloppy - 马虎的，草率的\n3. negligent - 疏忽的，大意的\n4. cursory - 草草的，匆忙的

<例句>:\n1. She is known for her meticulous attention to detail in her work.\n她在工作中以一丝不苟、关注细节而闻名。\n2. The auditor's report was meticulous, leaving no stone unturned.\n审计员的报告非常细致，没有遗漏任何细节。\n3. He is a meticulous planner, always making sure to consider every possible scenario.\n他是一个一丝不苟的计划者，总是确保考虑到所有可能的情况。\n4. Despite her meticulous preparations, she still managed to overlook a crucial detail.


In [10]:
response = await agent_all.run("请帮我把gorgeous存储到单词本中")

INFO - [Run][Start] FunctionAgent is about to start running with input:
请帮我把gorgeous存储到单词本中
INFO - [LLM][Start] ERNIEBot is about to start running with input:
 role: user 
 content: 请帮我把gorgeous存储到单词本中 
INFO - [LLM][End] ERNIEBot finished running with output:
 role: assistant 
 function_call: 
{
  "name": "ChatWithEB",
  "thoughts": "用户想要将gorgeous存储到单词本中，我需要调用ChatWithEB来获取gorgeous的详细信息，然后使用AddWordTool将其存储到单词本中。",
  "arguments": "{\"word\":\"gorgeous\"}"
} 
INFO - [Tool][Start] ChatWithEB is about to start running with input:
{
  "word": "gorgeous"
}
INFO - [Tool][End] ChatWithEB finished running with output:
{
  "response": "<单词翻译>: Gorgeous在中文中可翻译为“华美的”，“壮丽的”或者“宏伟的”。\n\n<近义词>:\n\n1. Magnificent\n2. Splendid\n3. Stunning\n4. Lavish\n5. Exquisite\n\n<反义词>:\n\n1. Plain\n2. Ugly\n3. Drab\n4. Unattractive\n5. Unsightly\n\n<例句>:\n\n1. The sunset was absolutely gorgeous, with a range of vivid colors spreading across the sky.\n2. She wore a gorgeous red dress that perfectly matched the color 

In [11]:
print(response.text)

已经为您添加了gorgeous这个单词，并提供了详细的解释和例句。gorgeous在中文中可翻译为“华美的”，“壮丽的”或者“宏伟的”。以下是它的详细解释和相关信息：

近义词：
1. Magnificent
2. Splendid
3. Stunning
4. Lavish
5. Exquisite

反义词：
1. Plain
2. Ugly
3. Drab
4. Unattractive
5. Unsightly

例句：
1. The sunset was absolutely gorgeous, with a range of vivid colors spreading across the sky.
2. She wore a gorgeous red dress that perfectly matched the color of her lips.
3. The castle looked gorgeous against the backdrop of the rolling green hills.
4. After years of renovation, the once dilapidated museum has been transformed into a gorgeous architectural masterpiece.
5. The gorgeous flowers in the garden attracted a variety of butterflies and bees.


In [12]:
add_word_tool.word_books

{'meticulous': "<单词翻译>: meticulous是“一丝不苟的，细致的，小心翼翼的”意思。\n\n<近义词>:\n1. thorough - 彻底的，详尽的\n2. careful - 小心的，仔细的\n3. precise - 精确的，严谨的\n4. attention to detail - 关注细节\n\n<反义词>:\n1. careless - 粗心的，不仔细的\n2. sloppy - 马虎的，草率的\n3. negligent - 疏忽的，大意的\n4. cursory - 草草的，匆忙的\n\n<例句>:\n1. She is known for her meticulous attention to detail in her work.\n她在工作中以一丝不苟、关注细节而闻名。\n2. The auditor's report was meticulous, leaving no stone unturned.\n审计员的报告非常细致，没有遗漏任何细节。\n3. He is a meticulous planner, always making sure to consider every possible scenario.\n他是一个一丝不苟的计划者，总是确保考虑到所有可能的情况。\n4. Despite her meticulous preparations, she still managed to overlook a crucial detail.",
 'gorgeous': '<单词翻译>: Gorgeous在中文中可翻译为“华美的”，“壮丽的”或者“宏伟的”。\n\n<近义词>:\n\n1. Magnificent\n2. Splendid\n3. Stunning\n4. Lavish\n5. Exquisite\n\n<反义词>:\n\n1. Plain\n2. Ugly\n3. Drab\n4. Unattractive\n5. Unsightly\n\n<例句>:\n\n1. The sunset was absolutely gorgeous, with a range of vivid colors spreading across the sky.\n2. She wore a gorgeous

## Agent其他相关方法
除了上述主要功能以外，接下来这个章节主要用于介绍一些其他可能会使用到的智能体功能。

### AgentResponse

AgentResponse作为智能体返回的结果，主要包含以下内容：
1. `text`: Agent在本Run中返回的最终文本
2. `steps`: Agent在本Run中执行的Step。
    含有tool_name（调用工具的名称），tool_args（工具的入参），input_files（输入的文件）， output_files（输出的文件）
    
    **Agent在步骤中涉及到的文件将存放在Step中，包括输入文件以及输出文件** 
3. `chat_history`: Agent在本Run中的对话历史
4. `status`: 包括`FINISHED`和`STOPPED`，其中`FINISHED`表示正常结束，`STOPPED`表示智能体因达到最大step数目强行终止

In [13]:
response.chat_history

[<HumanMessage role: 'user', content: '请帮我把gorgeous存储到单词本中', token_count: 1847>,
 <AIMessage role: 'assistant', function_call: {'name': 'ChatWithEB', 'thoughts': '用户想要将gorgeous存储到单词本中，我需要调用ChatWithEB来获取gorgeous的详细信息，然后使用AddWordTool将其存储到单词本中。', 'arguments': '{"word":"gorgeous"}'}, token_count: 48>,
 <FunctionMessage role: 'function', name: 'ChatWithEB', content: '{"response": "<单词翻译>: Gorgeous在中文中可翻译为“华美的”，“壮丽的”或者“宏伟的”。\\n\\n<近义词>:\\n\\n1. Magnificent\\n2. Splendid\\n3. Stunning\\n4. Lavish\\n5. Exquisite\\n\\n<反义词>:\\n\\n1. Plain\\n2. Ugly\\n3. Drab\\n4. Unattractive\\n5. Unsightly\\n\\n<例句>:\\n\\n1. The sunset was absolutely gorgeous, with a range of vivid colors spreading across the sky.\\n2. She wore a gorgeous red dress that perfectly matched the color of her lips.\\n3. The castle looked gorgeous against the backdrop of the rolling green hills.\\n4. After years of renovation, the once dilapidated museum has been transformed into a gorgeous architectural masterpiece.\\n5. The gorgeo

In [14]:
response.status

'FINISHED'

In [15]:
# Step为该Run中Agent所执行的相关操作，含有tool_name（调用工具的名称），tool_args（工具的入参），input_files（输入的文件）， output_files（输出的文件）
response.steps

[ToolStep(info={'tool_name': 'ChatWithEB', 'tool_args': '{"word":"gorgeous"}'}, result='{"response": "<单词翻译>: Gorgeous在中文中可翻译为“华美的”，“壮丽的”或者“宏伟的”。\\n\\n<近义词>:\\n\\n1. Magnificent\\n2. Splendid\\n3. Stunning\\n4. Lavish\\n5. Exquisite\\n\\n<反义词>:\\n\\n1. Plain\\n2. Ugly\\n3. Drab\\n4. Unattractive\\n5. Unsightly\\n\\n<例句>:\\n\\n1. The sunset was absolutely gorgeous, with a range of vivid colors spreading across the sky.\\n2. She wore a gorgeous red dress that perfectly matched the color of her lips.\\n3. The castle looked gorgeous against the backdrop of the rolling green hills.\\n4. After years of renovation, the once dilapidated museum has been transformed into a gorgeous architectural masterpiece.\\n5. The gorgeous flowers in the garden attracted a variety of butterflies and bees."}', input_files=[], output_files=[]),
 ToolStep(info={'tool_name': 'AddWordTool', 'tool_args': '{"word":"gorgeous","des":"<单词翻译>: Gorgeous在中文中可翻译为“华美的”，“壮丽的”或者“宏伟的”。\\n\\n<近义词>:\\n\\n1. Magnificent\\n2. Sple

### Tool以及Memory相关用法

In [16]:
# 获取当前所有工具
agent_all.get_tools()

[<name: ChatWithEB, description: ChatWithEB是一款根据用户的提供的单词，获取一个具体的单词描述、翻译以及例句的工具>,
 <name: AddWordTool, description: 添加单词以及它的详细解释到词库当中>,
 <name: pp-ocrv4/v2.2/ocr, server_url: https://tool-pp-ocrv4.aistudio-hub.baidu.com/, description: 使用PP-OCRv4模型进行光学字符识别。>]

In [17]:
# 查找某个工具
eb_tool = agent_all.get_tool("ChatWithEB")
# 取消某个工具
agent_all.unload_tool(eb_tool)
# 加载某个工具
agent_all.load_tool(eb_tool)

In [18]:
# 获取Agent的对话历史
agent_all.memory.get_messages()

[<HumanMessage role: 'user', content: '请帮我把图片中的单词进行解释后存储到单词本中\n<file>file-local-794791f0-a55e-11ee-b9cd-56c70f692df9</file>', token_count: 1668>,
 <AIMessage role: 'assistant', content: "您已经将单词“meticulous”添加到单词本中，以下是它的详细解释和相关信息：\n\n<单词翻译>: meticulous是“一丝不苟的，细致的，小心翼翼的”意思。\n\n<近义词>:\\n1. thorough - 彻底的，详尽的\\n2. careful - 小心的，仔细的\\n3. precise - 精确的，严谨的\\n4. attention to detail - 关注细节\n\n<反义词>:\\n1. careless - 粗心的，不仔细的\\n2. sloppy - 马虎的，草率的\\n3. negligent - 疏忽的，大意的\\n4. cursory - 草草的，匆忙的\n\n<例句>:\\n1. She is known for her meticulous attention to detail in her work.\\n她在工作中以一丝不苟、关注细节而闻名。\\n2. The auditor's report was meticulous, leaving no stone unturned.\\n审计员的报告非常细致，没有遗漏任何细节。\\n3. He is a meticulous planner, always making sure to consider every possible scenario.\\n他是一个一丝不苟的计划者，总是确保考虑到所有可能的情况。\\n4. Despite her meticulous preparations, she still managed to overlook a crucial detail.", token_count: 287>,
 <HumanMessage role: 'user', content: '请帮我把gorgeous存储到单词本中', token_count: 1847>,
 <AIMe

In [19]:
# 重置Agent的对话历史
agent_all.reset_memory()
agent_all.memory.get_messages()

[]

# 